---
---
# Use an ANN model to Predict the Wine's Sugar Content
### Goals:
1. Adapt code to predict sugar instead of quality.
2. Achieve as high of accuracy as possible for this problem.
### Notes:
- #### This differs from the previous projects, because now the model will predict sugar content instead of the subjective wine quality.
- #### Throughout this experiment, I'll be considering if the `quality` feature should be dropped. I would be interested to see if it harms or improves the model's performance.
- #### May still need to run a parametric experiment to figure out a good batch size. From previous projects I think batch sizes of 32 or 64 would be most stable.

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split

import numpy as np
import scipy.stats as stats

import pandas as pd

import matplotlib.pyplot as plt

import seaborn as sns